# Ham ya da Spam?

🎯 Bu görevin amacı, e-postaları **spam (1)** veya **normal e-posta (0)** olarak sınıflandırmaktır.

🧹 İlk olarak, bu metin verilerine **temizleme (cleaning)** teknikleri uygulanacaktır.

👩🏻‍🔬 Ardından, temizlenmiş metinler **sayısal bir gösterime** dönüştürülecektir.

✉️ Son olarak, her bir e-postayı spam mı yoksa normal mi olduğunu sınıflandırmak için  
***Multinomial Naive Bayes*** modeli uygulanacaktır.


## (0) NTLK kütüphanesi (Doğal Dil Araç Seti)

In [2]:
# !pip install nltk

In [3]:
# nltk'yi ilk kez içe aktarırken, birkaç yerleşik kütüphaneyi de indirmemiz gerekir.

import nltk

nltk.download('stopwords')
nltk.download('punkt')      # nltk<3.9.0 için
nltk.download('punkt_tab')  # nltk>=3.9.0 için
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/berkayturhan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/berkayturhan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/berkayturhan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/berkayturhan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/berkayturhan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
import pandas as pd

df = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/ham_spam_emails.csv")
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


## (1) (Metin) veri setinin temizlenmesi

Veri kümesi, ham [0] veya spam [1] olarak sınıflandırılan e-postalardan oluşur. Tahmin modelini eğitmeden önce veri kümesini temizlemeniz gerekir.

### (1.1) Noktalama İşaretlerini Kaldır

❓ Noktalama işaretlerini kaldırmak için bir işlev oluşturun. Bunu `text` sütununa uygulayın ve çıktıyı `clean_text` adlı veri çerçevesinin yeni bir sütununa ekleyin. ❓

In [5]:
import string

# Adım 1: Temizleme işlemini yapacak fonskiyonu tanımlıyoruz.
def remove_punctuation(text):
    # Metnin içindeki her bir harfi/karakteri (char) tek tek gezeriz.
    # Eğer bu karakter, string.punctuation (noktalama işaretleri) içinde YOKSA (not in) onu alırız.
    # "".join() ile de bu kalan harfleri tekrar birleştirip kelime/cümle haline getiririz.
    clean_text = "".join([char for char in text if char not in string.punctuation])
    return clean_text

# Adım 2: Fonksiyonu veri setine uygulama
# 'text' sütunundaki her bir satıra bu fonksiyonu uygula (apply) ve
# 'clean_text' adında yeni bir sütuna kaydet.
df['clean_text'] = df['text'].apply(remove_punctuation)

# Adım 3: Sonucu görmek için ilk 5 satırı yazdıralım
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,Subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,1,Subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,1,Subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,1,Subject 4 color printing special request addi...
4,"Subject: do not have money , get software cds ...",1,Subject do not have money get software cds fr...


### (1.2) Küçük Harf

❓ Metni küçük harfe çeviren bir işlev oluşturun. Bunu `clean_text`'e uygulayın ❓

In [6]:
# 1. Adım: Metni küçük harfe çevirelim
def lower_text(text):
    # .lower() metodu, python'da stringleri küçültmek için kullanılır.
    return text.lower()

# 2. Adım: Fonksiyonu 'clean_text' sütununa uygulayalım
# Dikkat: Artık işlemi bir önceki adımda oluşturduğumuz 'clean_text' sütunu üzerinde yapıyoruz.
df['clean_text'] = df['clean_text'].apply(lower_text)

# 3. Adım: Kontrol
df.head()


,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,1,subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,1,subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,1,subject 4 color printing special request addi...
4,"Subject: do not have money , get software cds ...",1,subject do not have money get software cds fr...


### (1.3) Sayıları Kaldır

❓ Metinden sayıları kaldırmak için bir işlev oluşturun. Bunu `clean_text`'e uygulayın ❓

In [7]:
# 1. Adım: Sayıları temizleyen fonksiyonu yazalım
def remove_numbers(text):
    # .isdigit() metodu, bir karakterin sayı olup olmadığını kontrol eder.
    # Eğer karakter sayı DEĞİLSE (not char.isdigit()), onu listeye ekle ve birleştir.
    text = "".join([char for char in text if not char.isdigit()])
    return text

# 2. Adım: Fonksiyonu 'clean_text' sütununa uygulayalım
df['clean_text'] = df['clean_text'].apply(remove_numbers)

# 3. Adım: Kontrol
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,1,subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,1,subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,1,subject color printing special request addit...
4,"Subject: do not have money , get software cds ...",1,subject do not have money get software cds fr...


### (1.4) StopWords'ü kaldırın

❓ Metinden durdurma kelimelerini kaldırmak için bir işlev oluşturun. Bunu `clean_text`'e uygulayın. ❓

In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# 1. Adım: Durdurma kelimelerini temizleyen fonksiyon
def remove_stopwords(text):
    # a. İngilizce stopword listesini yüklüyoruz (verimiz İngilizce olduğu için)
    stop_words = set(stopwords.words('english'))
    
    # b. Cümleyi kelimelerine ayırıyoruz (Tokenization)
    # "hello world" -> ["hello", "world"] olur.
    word_tokens = word_tokenize(text)
    
    # c. Eğer kelime stop_words listesinde YOKSA onu alıyoruz.
    # [w for w in word_tokens if not w in stop_words]
    filtered_text = [w for w in word_tokens if not w in stop_words]
    
    # d. Kelimeleri tekrar boşlukla birleştirip cümle haline getiriyoruz
    return " ".join(filtered_text)

# 2. Adım: Fonksiyonu uygulama
df['clean_text'] = df['clean_text'].apply(remove_stopwords)

# 3. Adım: Kontrol
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,subject naturally irresistible corporate ident...
1,Subject: the stock trading gunslinger fanny i...,1,subject stock trading gunslinger fanny merrill...
2,Subject: unbelievable new homes made easy im ...,1,subject unbelievable new homes made easy im wa...
3,Subject: 4 color printing special request add...,1,subject color printing special request additio...
4,"Subject: do not have money , get software cds ...",1,subject money get software cds software compat...


### (1.5) Lemmatize

❓ Metni lemmatize etmek için bir fonksiyon oluşturun. Çıktının bir kelime listesi değil, tek bir dize olduğundan emin olun. Bunu `clean_text`'e uygulayın. ❓

In [9]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# 1. Adım: Lemmatize fonksiyonunu tanımla
def lemmatize_text(text):
    # Lemmatizer nesnesini oluşturuyoruz (Araç kutusundan aleti çıkarıyoruz)
    lemmatizer = WordNetLemmatizer()
    
    # Cümleyi kelimelere bölüyoruz
    tokens = word_tokenize(text)
    
    # Her bir kelimeyi (token) tek tek köküne çeviriyoruz
    # lemmatizer.lemmatize(word) -> kelimenin kökünü verir
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Kelimeleri tekrar birleştirip tek bir string (yazı) yapıyoruz
    return " ".join(lemmatized_tokens)

# 2. Adım: Fonksiyonu uygula
df['clean_text'] = df['clean_text'].apply(lemmatize_text)

# 3. Adım: Kontrol
df.head()

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,subject naturally irresistible corporate ident...
1,Subject: the stock trading gunslinger fanny i...,1,subject stock trading gunslinger fanny merrill...
2,Subject: unbelievable new homes made easy im ...,1,subject unbelievable new home made easy im wan...
3,Subject: 4 color printing special request add...,1,subject color printing special request additio...
4,"Subject: do not have money , get software cds ...",1,subject money get software cd software compati...


## (2) Bag-of-Words Modellemesi

### (2.1) Metin verilerini sayılara dönüştürme

❓ `clean_text`'i varsayılan CountVectorizer ile Bag-of-Words temsiline vektörleştirin. `X_bow` olarak kaydedin. ❓

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

# 1. Adım: Vektörleştiriciyi (Sayma Makinesi) başlatıyoruz
# Bu araç, kelimeleri saymak için ayarlanmış bir robot gibidir.
vectorizer = CountVectorizer()

# 2. Adım: Veriyi makineye verip dönüştürüyoruz (fit & transform)
# fit: Tüm kelime havuzunu (sözlüğü) öğrenir.
# transform: Her satırdaki kelimeleri sayıp matrise çevirir.
X_bow = vectorizer.fit_transform(df['clean_text'])

# 3. Adım: Sonucu kontrol edelim (Boyutlarına bakalım)
# Çıktı (Satır Sayısı, Kelime Sayısı) şeklinde olacak.
print(X_bow.shape)

(5728, 30988)


### (2.2) Çok terimli Naive Bayes Modellemesi

❓ MultinomialNB modelini bag-of-words verileriyle çapraz doğrulayın. Modelin doğruluğunu puanlayın. ❓

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

# 1. Adım: Hedef değişkenimizi (y) belirliyoruz.
# Neyin tahmin edileceğini söylüyoruz: 'spam' sütunu (0 veya 1 değerleri)
y = df['spam']

# 2. Adım: Modeli tanımlıyoruz (Multinomial Naive Bayes)
nb_model = MultinomialNB()

# 3. Adım: Çapraz Doğrulama (Cross Validation) ile modeli zorluyoruz
# nb_model: Modelimiz
# X_bow: Ders notları (Kelimeler)
# y: Cevap anahtarı (Spam mi değil mi?)
# cv=5: Sınavı 5 kere tekrarla
scores = cross_val_score(nb_model, X_bow, y, cv=5)

# 4. Adım: Sonucu yazdırma
# 5 sınavın ortalamasını alıyoruz.
print("Doğruluk Skoru (Accuracy):", scores.mean())

Doğruluk Skoru (Accuracy): 0.9895252901681946


🏁 Tebrikler!

💾 Not defterinizi git add/commit/push yapmayı unutmayın...

🚀 ... ve bir sonraki challenge'a geçin!